In [40]:
import os
import time

# from dotenv import load_dotenv
import requests
import json
import re
from datetime import date
import csv
import pandas as pd


#load_dotenv()

API_KEY="bc7791c727dae92fcf26b6e1338575e9"
HISTORICAL_API_KEY="668b5200589eb79da855b2f68cd0749a"
BASE = "https://api.the-odds-api.com/v4"


In [51]:
class Collector:

    sports_titles = []
    sports_keys = []

    @staticmethod
    def getAPIKey():
        print(HISTORICAL_API_KEY)


    @staticmethod
    def getEvents(sport):
        url = f'{BASE}/sports/{sport}/events?apiKey={API_KEY}'

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Events Data. Status code: {response.status_code}")
            data = None

        if data is not None:
            with open(f"{sport}-event-{date.today()}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

    @staticmethod
    def getHistoricalEvents(sport, date):

        # ISO format: 2021-10-18T12:00:00Z
        # yyyy mm dd

        if not Collector.validateDate(date):
            print(f"Invalid date: {date}")
            return

        orig_date = date
        date += "T10:00:00Z"  # auto collect at 10 AM

        url = f"{BASE}/historical/sports/{sport}/events?apiKey={HISTORICAL_API_KEY}&date={date}"

        print(url)

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Events Data. Status code: {response.status_code}")
            data = None
            return None

        # print(data)

        if data is not None:
            with open(f"basketball-events/{sport}-event-{orig_date}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

        return "working"


    @staticmethod
    def getPropByEventFiles(dir, sport, markets, regions):
        # gets all props for every single event in basketball-events
        for file in os.listdir(dir):
            if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
                with open(os.path.join(dir, file), "r") as f:
                    data = json.load(f)

                timestamp = data["timestamp"]

                for event in data["data"]:
                    eventId = event["id"]
                    Collector.getHistoricalNbaPropsByEventId(sport, eventId, markets, regions, timestamp, True)

    @staticmethod
    def convertPropFilesToSingularCSV(dir):
        csv_player_data = []

        for file in os.listdir(dir):
            if os.path.isfile(os.path.join(dir, file)) and file.endswith('.json'):
                with open(os.path.join(dir, file), "r") as f:
                    data = json.load(f)

                filename = os.path.join(dir, file)
                date = filename.split("props-")[1].replace(".json", "")


                data = data["data"]

                player_data = {}

                for market in data["bookmakers"][0]["markets"]:
                    key = market["key"]

                    for odds in market["outcomes"]:
                        player = odds["description"]
                        if player not in player_data:
                            player_data[player] = {}

                        if key not in player_data[player]:
                            player_data[player][key] = {}

                        player_data[player][key]["line"] = odds["point"]
                        player_data[player][key][odds["name"].lower()] = odds["price"]


                for player, lines in player_data.items():
                    row = {'Player': player, 'Date': date}
                    for prop_type, values in lines.items():
                        for prop_info, value in values.items():
                            row[f"{prop_type}_{prop_info}"] = value
                    csv_player_data.append(row)


        headers = set()
        for row in csv_player_data:
            headers.update(row.keys())

        # Write CSV data to file
        with open('odds_output.csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=sorted(headers))
            writer.writeheader()
            writer.writerows(csv_player_data)


    @staticmethod
    def getHistoricalNBAPropsByEventId(sport, eventId, markets, regions, date, testing):
        if (testing == True):
            with open("basketball-player-props/da359da99aa27e97d38f2df709343998-props-2023-11-29T10:00:00Z.json", "r") as file:
                d = json.loads(file)
                return d

        # ISO format: 2021-10-18T12:00:00Z
        # yyyy mm dd

        if not Collector.validateDate(date):
            print(f"Invalid date: {date}")
            return

        orig_date = date
        date += "T10:00:00Z" # auto collect at 10 AM


        # /v4/historical/sports/{sport}/events/{eventId}/odds?apiKey={apiKey}&regions={regions}&markets={markets}&dateFormat={dateFormat}&oddsFormat={oddsFormat}&date={date}
        url = f'{BASE}/historical/sports/{sport}/events/{eventId}/odds?apiKey={HISTORICAL_API_KEY}&regions={",".join(regions)}&markets={",".join(markets)}&date={date}'

        print(url)
        response = requests.request("GET", url, headers={}, data={})
        print(response.json())


        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Player Props Data for {eventId}. Status code: {response.status_code}")
            data = None

            return None


        if data is not None:
            with open(f"basketball-player-props/{eventId}-props-{orig_date}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)

        return data



    @staticmethod
    def getNBAPropsByEventId(sport, eventId, markets, regions):
        url = f'{BASE}/sports/{sport}/events/{eventId}/odds?markets={",".join(markets)}&regions={",".join(regions)}&apiKey={API_KEY}'

        response = requests.request("GET", url, headers={}, data={})

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Error Collecting Player Props Data for {eventId}. Status code: {response.status_code}")
            data = None

        if data is not None:
            with open(f"{eventId}-props-{date.today()}.json", "w") as json_file:
                json.dump(data, json_file, indent=4)


    @staticmethod
    def exploreHistoricalPropData(filename):
        date = filename.split("props-")[1].replace(".json", "")

        with open(filename) as f:
            data = json.load(f)

        data = data["data"]

        player_data = {}

        for market in data["bookmakers"][0]["markets"]:
            key = market["key"]

            for odds in market["outcomes"]:
                player = odds["description"]
                if player not in player_data:
                    player_data[player] = {}

                if key not in player_data[player]:
                    player_data[player][key] = {}

                player_data[player][key]["line"] = odds["point"]
                player_data[player][key][odds["name"].lower()] = odds["price"]


        csv_player_data = []
        for player, lines in player_data.items():
            row = {'Player': player, 'Date': date}
            for prop_type, values in lines.items():
                for prop_info, value in values.items():
                    row[f"{prop_type}_{prop_info}"] = value
            csv_player_data.append(row)


        headers = set()
        for row in csv_player_data:
            headers.update(row.keys())

        # Write CSV data to file
        with open('output.csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=sorted(headers))
            writer.writeheader()
            writer.writerows(csv_player_data)


    @staticmethod
    def validateRegion(reg):
        regex_pattern = r'^(us|us2|uk|au|eu)(,(us|us2|uk|au|eu))*$'
        return bool(re.match(regex_pattern, reg))

    @staticmethod
    def validateMarket(market):
        regex_pattern = r'^(h2h|spreads|totals|outrights|h2h_lay|outrights_lay)(,(h2h|spreads|totals|outrights|h2h_lay|outrights_lay))*$'
        return bool(re.match(regex_pattern, market))

    @staticmethod
    def validateDate(date):
        regex_pattern = r'^\d{4}-\d{2}-\d{2}$'
        return bool(re.match(regex_pattern, date))

Get all NBA events

In [42]:
# Code checks how many unique game dates there are

df = pd.read_csv('COMPLETE_player_data.csv')

unique_dates = df['Date'].value_counts()

unique_dates = unique_dates.sort_index()

filtered_dates = []

for date, count in unique_dates.items():
    spdate = date.split("-")
    if (spdate[1] == "12" and int(spdate[2]) > 12) or (spdate[1] == "01") :
        filtered_dates.append((date, count))
print(filtered_dates)


[('2023-12-13', 105), ('2023-12-14', 71), ('2023-12-15', 90), ('2023-12-16', 105), ('2023-12-17', 45), ('2023-12-18', 128), ('2023-12-19', 37), ('2023-12-20', 116), ('2023-12-21', 81), ('2023-12-22', 69), ('2023-12-23', 137), ('2023-12-25', 46), ('2023-12-26', 89), ('2023-12-27', 60), ('2023-12-28', 74), ('2023-12-29', 101), ('2023-12-30', 59), ('2023-12-31', 57), ('2024-01-01', 81), ('2024-01-02', 56), ('2024-01-03', 124), ('2024-01-04', 18), ('2024-01-05', 140), ('2024-01-06', 43), ('2024-01-07', 89), ('2024-01-08', 58), ('2024-01-09', 51), ('2024-01-10', 103), ('2024-01-11', 51), ('2024-01-12', 94), ('2024-01-13', 69), ('2024-01-14', 48), ('2024-01-15', 111), ('2024-01-16', 29), ('2024-01-17', 91), ('2024-01-18', 54), ('2024-01-19', 78), ('2024-01-20', 89), ('2024-01-21', 64), ('2024-01-22', 85), ('2024-01-23', 60), ('2024-01-24', 73), ('2024-01-25', 82), ('2024-01-26', 81), ('2024-01-27', 105), ('2024-01-28', 56), ('2024-01-29', 120), ('2024-01-30', 55), ('2024-01-31', 101)]


In [46]:
Collector.getHistoricalEvents("basketball_nba", filtered_dates[0][0])

https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-13T10:00:00Z
{'timestamp': '2023-12-13T09:55:38Z', 'previous_timestamp': '2023-12-13T09:50:38Z', 'next_timestamp': '2023-12-13T10:00:39Z', 'data': [{'id': 'ce6c9aa55ee8613be3168bf5031920bf', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:10:00Z', 'home_team': 'Detroit Pistons', 'away_team': 'Philadelphia 76ers'}, {'id': '532e7f4618fc67f69e19b99394ed0d06', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:10:00Z', 'home_team': 'Washington Wizards', 'away_team': 'New Orleans Pelicans'}, {'id': '62b3bd979a162d755689d1f073281ba4', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-12-14T00:40:00Z', 'home_team': 'Toronto Raptors', 'away_team': 'Atlanta Hawks'}, {'id': 'd73b5a32c4fed2c803e8a0b60cb8cfaa', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time':

In [49]:
sport = "basketball_nba"

In [52]:
import time

In [54]:
print(len(filtered_dates))

49


In [55]:
for i in range(5, len(filtered_dates)):
    print(i)
    resp = Collector.getHistoricalEvents(sport, filtered_dates[i][0])
    if resp is None:
        break
    time.sleep(1.01)

5
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-18T10:00:00Z
6
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-19T10:00:00Z
7
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-20T10:00:00Z
8
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-21T10:00:00Z
9
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-22T10:00:00Z
10
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-23T10:00:00Z
11
https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey=668b5200589eb79da855b2f68cd0749a&date=2023-12-25T10:00:00Z
12
https://api.the